## -----------------------------------------------------------------
# AI & ML - Coursework 1 - 1st diet
## Student Name: Akama Emmanuel Ovokerie
## Student ID: S2229758

ATTESTATION: I confirm that the material contained within the submitted coursework is all my own work 
## -----------------------------------------------------------------

## 1. Introduction and problem definition
This report contained in this coursework is focused on implementing the first six (6) steps of the machine learning pipeline, problem definition, data ingestion, data preparation, data segregation, model training and model evaluation.

The dataset to be analyzed contains 9358 instances of hourly-averaged responses from an array of 5 metal-oxide chemical sensors embedded in an air quality chemical multi-sensor device. The devices were located on the field in a significantly polluted area, at road level, within an Italian city. The data contain observations from March 2004 to February 2005 (one year).

The problem definition for this report are a follows:
1. Predict the CO concentration (in mg/m3) based on, at least, the PT08.S1(CO) raw sensor readings, day of the week and time. Maybe temperature and humidity can play a role as well? Use CO(GT) as the ground truth.
2. Define your own Air Quality Index by combining the ground-truth readings of several gases. Then, use ML to predict your defined Air Quality Index from several raw sensor readings and other columns of interest (obviously without using the ground truth columns).

### Module Imports
To execute the tasks contained in this report, we need to import relevant Python modules. Since it is considered a good practice to import modules at the top of the notebook file, we shall do so here, rather than spread it throughout the whole file. This way a single look will inform of all the required modules to execute all code cells from one single cell

#TODO - Import the relevant python modules

In [1]:
# import python libraries
import numpy as np
import pandas as pd

# import more libraries (specific to task 1 and 2)
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from math import sqrt

## 2. Data ingeston

Data ingestion is the process of moving data from a source into a landing area or an object store where it can be used for ad hoc queries and analytics. 
A simple data ingestion pipeline consumes data from a point of origin, cleans it up a bit, then writes it to a destination.

TO-DO: In this section, we shall
1. Ingest the dataset saved locally with [pandas](https://www.w3schools.com/python/pandas/pandas_csv.asp) *read_csv()* library method
2. Create a data dictionary to
    - identify the target and dependent variables
    - record the name, type and category of each column object

In [6]:
# load the csv file from local storage
# NB: change the source path if loading from another location
df = pd.read_csv('C:/Users/emman/OneDrive - GLASGOW CALEDONIAN UNIVERSITY/Documents/GCU/Module - AI & ML/Assessments/CW 01/Report/Dataset/AirQuality/AirQuality.csv')

# preview the data
df.head(10)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888,NaN,NaN
5,10/03/2004,23:00:00,1.2,1197,38,4.7,750,89,1337,96,1393,949,11.2,59.2,0.7848,NaN,NaN
6,11/03/2004,00:00:00,1.2,1185,31,3.6,690,62,1462,77,1333,733,11.3,56.8,0.7603,NaN,NaN
7,11/03/2004,01:00:00,1.0,1136,31,3.3,672,62,1453,76,1333,730,10.7,60.0,0.7702,NaN,NaN
8,11/03/2004,02:00:00,0.9,1094,24,2.3,609,45,1579,60,1276,620,10.7,59.7,0.7648,NaN,NaN
9,11/03/2004,03:00:00,0.6,1010,19,1.7,561,-200,1705,-200,1235,501,10.3,60.2,0.7517,NaN,NaN


The data dictionary schema defination for the target and dependent column attributes are given below;

1. Independent (or Target) Variable(s)
    - Task 1 - *CO(GT) [Float64, Numerical, Continuous]* - Total hourly-averaged concentrations for Carbon Monoxide (CO, in mg/m3)*
    - Task 2 - *Air Quality Index [Int64, Categorical, Ordinal]* - Air quality index based on the average ground-truth readings of several gases

2. Dependent Variable(s)
    - *Date - [Object, Categorical, Not ordinal]* - The date of data record. This can be further classified as 'Weekday' or 'Weekend'
    - *Time - [Object, Categorical, Ordinal]* - The time of data record. This can be further classified into 'Peak' or 'Off-peak' periods
    - *PT08.S1(CO) - [Int64, Numerical, Discrete]* - Estimated hourly-averaged concentrations for Carbon Monoxide (CO, in mg/m3) from the PT08.S1(CO) sensor
    - *NMHC(GT) - [Int64, Numerical, Discrete]* - Total hourly-averaged concentrations for Non-Methanic Hydrocarbons (NMHC, in μg/m3) 
    - *C6H6(GT) - [Float64, Numerical, Continuous]* - Total hourly-averaged concentrations for Benzene (C6H6, in mg/m3) 
    - *PT08.S2(NMHC) - [Int64, Numerical, Discrete]* - Estimated hourly-averaged concentrations for Non-Methanic Hydrocarbons (NMHC, in μg/m3) from the PT08.S2(NMHC) sensor
    - *NOx(GT) - [Int64, Numerical, Discrete]* - Total hourly-averaged concentrations for Nitrogen Oxides (NOx, in μg/m3)
    - *PT08.S3(NOx) - [Int64, Numerical, Discrete]* - Estimated hourly-averaged concentrations for Nitrogen Oxides (NOx, in μg/m3) from the PT08.S3(NOx) sensor
    - *NO2(GT) - [Int64, Numerical, Discrete]* - Total hourly-averaged concentrations for Nitrogen Dioxide (NO2, in mg/m3) 
    - *PT08.S4(NO2) - [Int64, Numerical, Discrete]* - Estimated hourly-averaged concentrations for Nitrogen Dioxides (NO2, in μg/m3) from the PT08.S4(NO2) sensor
    - *PT08.S5(O3) - [Int64, Numerical, Discrete]* - Estimated hourly-averaged concentrations for Ozone (O3, in μg/m3) from the PT08.S5(O3) sensor
    - *T(C) - [Float64, Numerical, Continuous]* - Hourly-averaged temperature reading (in degree centigrade). 
    - *RH - [Float64, Numerical, Continuous]* - Hourly-averaged relative humidity reading. 
    - *AH - [Float64, Numerical, Continuous]* - Hourly-averaged absolute humidity reading. 

Note: Some features of the dataset were represented as ground truths while others were represented as estimates. Gound truth represent exact values. In this case, these could be values obtained from preliminary exploratory data analysis already performed on the dataset. In the dataset, the following features were represented as ground truths, Date, Time, CO(GT), NMHC(GT), C6H6(GT), NOx(GT), and NO2(GT), T(C), RH, and AH. The remaining features [PT08.S1(CO), PT08.S2(NMHC), PT08.S3(NOx), PT08.S4(NO2), PT08.S5(O3)] were captured in the dataset as estimates. These estimated values captured by the measuring equipment.

In [7]:
# show column types
print(df.dtypes)

Date              object
Time              object
CO(GT)           float64
PT08.S1(CO)        int64
NMHC(GT)           int64
C6H6(GT)         float64
PT08.S2(NMHC)      int64
NOx(GT)            int64
PT08.S3(NOx)       int64
NO2(GT)            int64
PT08.S4(NO2)       int64
PT08.S5(O3)        int64
T(C)             float64
RH               float64
AH               float64
Unnamed: 15      float64
Unnamed: 16      float64
dtype: object


In [8]:
# show complete information  
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9357 entries, 0 to 9356
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           9357 non-null   object 
 1   Time           9357 non-null   object 
 2   CO(GT)         9357 non-null   float64
 3   PT08.S1(CO)    9357 non-null   int64  
 4   NMHC(GT)       9357 non-null   int64  
 5   C6H6(GT)       9357 non-null   float64
 6   PT08.S2(NMHC)  9357 non-null   int64  
 7   NOx(GT)        9357 non-null   int64  
 8   PT08.S3(NOx)   9357 non-null   int64  
 9   NO2(GT)        9357 non-null   int64  
 10  PT08.S4(NO2)   9357 non-null   int64  
 11  PT08.S5(O3)    9357 non-null   int64  
 12  T(C)           9357 non-null   float64
 13  RH             9357 non-null   float64
 14  AH             9357 non-null   float64
 15  Unnamed: 15    0 non-null      float64
 16  Unnamed: 16    0 non-null      float64
dtypes: float64(7), int64(8), object(2)
memory usage: 1.2

## 3. Data preparation (common to both tasks)
A critical step in data preparation is data cleaning.
This step typically involves fixing bad data in the dataset.

Bad data could be represented in the following form; 
- missing or empty cells
- data in the wrong format
- wrong or misplaced data
- duplicate records

Data cleaning corrects errors and fills in missing data as a step to ensure data quality.

*[ Source: w3schools - https://www.w3schools.com/python/pandas/pandas_cleaning.asp ]*

TO-DO: In this section, we shall
1. Delete empty or irrelevant columns
2. Replace missing or invalid values (For example, -200 with NaN)
3. Create a new column to capture the day of the week, along with peak and off-peak periods
4. Group related rows into bins represented as aggregated values (where possible)

In [9]:
# drop empty columns (including Unnamed 15 and 16)
df.dropna(how="all", axis=1, inplace=True)

# preview the data
df.head(10)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH
0,10/03/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,10/03/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,10/03/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,10/03/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,10/03/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
5,10/03/2004,23:00:00,1.2,1197,38,4.7,750,89,1337,96,1393,949,11.2,59.2,0.7848
6,11/03/2004,00:00:00,1.2,1185,31,3.6,690,62,1462,77,1333,733,11.3,56.8,0.7603
7,11/03/2004,01:00:00,1.0,1136,31,3.3,672,62,1453,76,1333,730,10.7,60.0,0.7702
8,11/03/2004,02:00:00,0.9,1094,24,2.3,609,45,1579,60,1276,620,10.7,59.7,0.7648
9,11/03/2004,03:00:00,0.6,1010,19,1.7,561,-200,1705,-200,1235,501,10.3,60.2,0.7517


In [10]:
# replace invalid entries (use NaN for -200)
df[df.isin([-200])] = np.nan

# preview the data
df.head(10)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH
0,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
5,10/03/2004,23:00:00,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848
6,11/03/2004,00:00:00,1.2,1185.0,31.0,3.6,690.0,62.0,1462.0,77.0,1333.0,733.0,11.3,56.8,0.7603
7,11/03/2004,01:00:00,1.0,1136.0,31.0,3.3,672.0,62.0,1453.0,76.0,1333.0,730.0,10.7,60.0,0.7702
8,11/03/2004,02:00:00,0.9,1094.0,24.0,2.3,609.0,45.0,1579.0,60.0,1276.0,620.0,10.7,59.7,0.7648
9,11/03/2004,03:00:00,0.6,1010.0,19.0,1.7,561.0,NaN,1705.0,NaN,1235.0,501.0,10.3,60.2,0.7517


In [11]:
# add column to capture the days of the week
df['DayOfWeek'] = pd.to_datetime(df['Date'], dayfirst=True).dt.day_name()

# preview the data
df['DayOfWeek'].head(10)

0    Wednesday
1    Wednesday
2    Wednesday
3    Wednesday
4    Wednesday
5    Wednesday
6     Thursday
7     Thursday
8     Thursday
9     Thursday
Name: DayOfWeek, dtype: object

In [12]:
# add columns to flag 'DayOfWeek' as 'Weekday' or 'Weekend'
df['Weekday'] = df['DayOfWeek'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
df['Weekend'] = df['DayOfWeek'].isin(['Saturday', 'Sunday'])

# preview the data
df.head(10)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH,DayOfWeek,Weekday,Weekend
0,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,Wednesday,True,False
1,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,Wednesday,True,False
2,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,Wednesday,True,False
3,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,Wednesday,True,False
4,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,Wednesday,True,False
5,10/03/2004,23:00:00,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848,Wednesday,True,False
6,11/03/2004,00:00:00,1.2,1185.0,31.0,3.6,690.0,62.0,1462.0,77.0,1333.0,733.0,11.3,56.8,0.7603,Thursday,True,False
7,11/03/2004,01:00:00,1.0,1136.0,31.0,3.3,672.0,62.0,1453.0,76.0,1333.0,730.0,10.7,60.0,0.7702,Thursday,True,False
8,11/03/2004,02:00:00,0.9,1094.0,24.0,2.3,609.0,45.0,1579.0,60.0,1276.0,620.0,10.7,59.7,0.7648,Thursday,True,False
9,11/03/2004,03:00:00,0.6,1010.0,19.0,1.7,561.0,NaN,1705.0,NaN,1235.0,501.0,10.3,60.2,0.7517,Thursday,True,False


In [13]:
# add column to check peak or off-peak hours (initialized to False)
df['IsPeak'] = False 

# update 'IsPeak' based on the day and time
df.loc[((df['Time'] >= '08:00:00') & (df['Time'] <= '12:00:00') & df['Weekday'] == True), 'IsPeak'] = True
df.loc[((df['Time'] >= '18:00:00') & (df['Time'] <= '22:00:00') & df['Weekday'] == True), 'IsPeak'] = True
df.loc[((df['Time'] >= '02:00:00') & (df['Time'] <= '06:00:00') & df['Weekend'] == True), 'IsPeak'] = True

# preview the data
df.head(10)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH,DayOfWeek,Weekday,Weekend,IsPeak
0,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,Wednesday,True,False,True
1,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,Wednesday,True,False,True
2,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,Wednesday,True,False,True
3,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,Wednesday,True,False,True
4,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,Wednesday,True,False,True
5,10/03/2004,23:00:00,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848,Wednesday,True,False,False
6,11/03/2004,00:00:00,1.2,1185.0,31.0,3.6,690.0,62.0,1462.0,77.0,1333.0,733.0,11.3,56.8,0.7603,Thursday,True,False,False
7,11/03/2004,01:00:00,1.0,1136.0,31.0,3.3,672.0,62.0,1453.0,76.0,1333.0,730.0,10.7,60.0,0.7702,Thursday,True,False,False
8,11/03/2004,02:00:00,0.9,1094.0,24.0,2.3,609.0,45.0,1579.0,60.0,1276.0,620.0,10.7,59.7,0.7648,Thursday,True,False,False
9,11/03/2004,03:00:00,0.6,1010.0,19.0,1.7,561.0,NaN,1705.0,NaN,1235.0,501.0,10.3,60.2,0.7517,Thursday,True,False,False


In [14]:
# update 'DayOfWeek' with numerical equivalents
df.loc[df['DayOfWeek'] == 'Monday', 'DayOfWeek'] = 1
df.loc[df['DayOfWeek'] == 'Tuesday', 'DayOfWeek'] = 2
df.loc[df['DayOfWeek'] == 'Wednesday', 'DayOfWeek'] = 3
df.loc[df['DayOfWeek'] == 'Thursday', 'DayOfWeek'] = 4
df.loc[df['DayOfWeek'] == 'Friday', 'DayOfWeek'] = 5
df.loc[df['DayOfWeek'] == 'Saturday', 'DayOfWeek'] = 6
df.loc[df['DayOfWeek'] == 'Sunday', 'DayOfWeek'] = 7

# preview the data
df.head(10) 

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH,DayOfWeek,Weekday,Weekend,IsPeak
0,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,3,True,False,True
1,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,3,True,False,True
2,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,3,True,False,True
3,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,3,True,False,True
4,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,3,True,False,True
5,10/03/2004,23:00:00,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848,3,True,False,False
6,11/03/2004,00:00:00,1.2,1185.0,31.0,3.6,690.0,62.0,1462.0,77.0,1333.0,733.0,11.3,56.8,0.7603,4,True,False,False
7,11/03/2004,01:00:00,1.0,1136.0,31.0,3.3,672.0,62.0,1453.0,76.0,1333.0,730.0,10.7,60.0,0.7702,4,True,False,False
8,11/03/2004,02:00:00,0.9,1094.0,24.0,2.3,609.0,45.0,1579.0,60.0,1276.0,620.0,10.7,59.7,0.7648,4,True,False,False
9,11/03/2004,03:00:00,0.6,1010.0,19.0,1.7,561.0,NaN,1705.0,NaN,1235.0,501.0,10.3,60.2,0.7517,4,True,False,False


In [15]:
# also, update 'IsPeak' with binary equivalents
df.loc[df['IsPeak'] == False, 'IsPeak'] = 0
df.loc[df['IsPeak'] == True, 'IsPeak'] = 1

# preview the data
df.head(10)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T(C),RH,AH,DayOfWeek,Weekday,Weekend,IsPeak
0,10/03/2004,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,3,True,False,1
1,10/03/2004,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,3,True,False,1
2,10/03/2004,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,3,True,False,1
3,10/03/2004,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,3,True,False,1
4,10/03/2004,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,3,True,False,1
5,10/03/2004,23:00:00,1.2,1197.0,38.0,4.7,750.0,89.0,1337.0,96.0,1393.0,949.0,11.2,59.2,0.7848,3,True,False,0
6,11/03/2004,00:00:00,1.2,1185.0,31.0,3.6,690.0,62.0,1462.0,77.0,1333.0,733.0,11.3,56.8,0.7603,4,True,False,0
7,11/03/2004,01:00:00,1.0,1136.0,31.0,3.3,672.0,62.0,1453.0,76.0,1333.0,730.0,10.7,60.0,0.7702,4,True,False,0
8,11/03/2004,02:00:00,0.9,1094.0,24.0,2.3,609.0,45.0,1579.0,60.0,1276.0,620.0,10.7,59.7,0.7648,4,True,False,0
9,11/03/2004,03:00:00,0.6,1010.0,19.0,1.7,561.0,NaN,1705.0,NaN,1235.0,501.0,10.3,60.2,0.7517,4,True,False,0


In [16]:
# group by 'Date' and 'IsPeak', then compute median values
# NB: median values are used to reduce the effect of outliers
df = df.groupby(by=['Date', 'IsPeak']).median(numeric_only=True)

# preview the data
df.head(10)

CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  \
Date       IsPeak                                                           
01/01/2005 0         2.15       1102.0       NaN      7.70          885.0   
           1         1.90       1054.0       NaN      5.60          791.0   
01/02/2005 0         1.45       1048.5       NaN      5.10          769.5   
           1         4.05       1379.5       NaN     17.10         1213.0   
01/03/2005 0         0.60        772.5       NaN      1.10          507.5   
           1         1.45        893.5       NaN      5.10          770.0   
01/04/2004 0         1.70       1125.0     127.0      6.30          825.0   
           1         3.50       1385.0     482.0     15.95         1171.0   
01/04/2005 0         1.00        892.0       NaN      2.65          627.5   
           1         1.20        933.5       NaN      3.85          703.5   

                   NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  \
Date       IsPeak                                                              
01/01/2005 0         297.0         765.0    133.5        1051.0       1206.0   
           1         217.0         830.0    116.0         967.0       1131.0   
01/02/2005 0         241.0         800.0    146.0         957.0       1272.5   
           1         708.5         531.0    187.5        1299.0       1703.0   
01/03/2005 0         134.0        1344.0     84.0         647.0        317.5   
           1         297.0         917.5    132.5         805.5        629.5   
01/04/2004 0         102.0        1002.0     73.0        1425.0        944.0   
           1         201.5         750.0    121.5        1811.5       1340.5   
01/04/2005 0         147.5         974.0    100.0         926.5        461.5   
           1         190.0         852.0    125.0         969.0        574.5   

                    T(C)     RH       AH  Weekday  Weekend  
Date       IsPeak                                           
01/01/2005 0        7.80  51.80  0.48870      0.0      1.0  
           1        4.30  55.30  0.46890      0.0      1.0  
01/02/2005 0        4.15  49.75  0.47685      1.0      0.0  
           1        7.30  47.30  0.49525      1.0      0.0  
01/03/2005 0        0.50  37.45  0.24015      1.0      0.0  
           1        1.60  35.20  0.23880      1.0      0.0  
01/04/2004 0       11.00  63.90  0.83940      1.0      0.0  
           1       18.75  40.25  0.87180      1.0      0.0  
01/04/2005 0       14.55  43.35  0.71310      1.0      0.0  
           1       16.10  31.75  0.56805      1.0      0.0

In [17]:
# drop irrelevant columns, 'Weekday' and 'Weekend'
df = df.drop(['Weekday', 'Weekend'], axis=1)

# show complete information  
print(df.info())

# preview the data
df.head(10)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 782 entries, ('01/01/2005', 0) to ('31/12/2004', 1)
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CO(GT)         702 non-null    float64
 1   PT08.S1(CO)    764 non-null    float64
 2   NMHC(GT)       87 non-null     float64
 3   C6H6(GT)       764 non-null    float64
 4   PT08.S2(NMHC)  764 non-null    float64
 5   NOx(GT)        709 non-null    float64
 6   PT08.S3(NOx)   764 non-null    float64
 7   NO2(GT)        709 non-null    float64
 8   PT08.S4(NO2)   764 non-null    float64
 9   PT08.S5(O3)    764 non-null    float64
 10  T(C)           764 non-null    float64
 11  RH             764 non-null    float64
 12  AH             764 non-null    float64
dtypes: float64(13)
memory usage: 84.9+ KB
None


CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  \
Date       IsPeak                                                           
01/01/2005 0         2.15       1102.0       NaN      7.70          885.0   
           1         1.90       1054.0       NaN      5.60          791.0   
01/02/2005 0         1.45       1048.5       NaN      5.10          769.5   
           1         4.05       1379.5       NaN     17.10         1213.0   
01/03/2005 0         0.60        772.5       NaN      1.10          507.5   
           1         1.45        893.5       NaN      5.10          770.0   
01/04/2004 0         1.70       1125.0     127.0      6.30          825.0   
           1         3.50       1385.0     482.0     15.95         1171.0   
01/04/2005 0         1.00        892.0       NaN      2.65          627.5   
           1         1.20        933.5       NaN      3.85          703.5   

                   NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  \
Date       IsPeak                                                              
01/01/2005 0         297.0         765.0    133.5        1051.0       1206.0   
           1         217.0         830.0    116.0         967.0       1131.0   
01/02/2005 0         241.0         800.0    146.0         957.0       1272.5   
           1         708.5         531.0    187.5        1299.0       1703.0   
01/03/2005 0         134.0        1344.0     84.0         647.0        317.5   
           1         297.0         917.5    132.5         805.5        629.5   
01/04/2004 0         102.0        1002.0     73.0        1425.0        944.0   
           1         201.5         750.0    121.5        1811.5       1340.5   
01/04/2005 0         147.5         974.0    100.0         926.5        461.5   
           1         190.0         852.0    125.0         969.0        574.5   

                    T(C)     RH       AH  
Date       IsPeak                         
01/01/2005 0        7.80  51.80  0.48870  
           1        4.30  55.30  0.46890  
01/02/2005 0        4.15  49.75  0.47685  
           1        7.30  47.30  0.49525  
01/03/2005 0        0.50  37.45  0.24015  
           1        1.60  35.20  0.23880  
01/04/2004 0       11.00  63.90  0.83940  
           1       18.75  40.25  0.87180  
01/04/2005 0       14.55  43.35  0.71310  
           1       16.10  31.75  0.56805

Before we proceeed with further specific steps to prepare the data, we need to review a few statistical attributes about the data, including the following;
 - Total count
 - Minimum and maximum values
 - Mean, standard deviation, and interquatile ranges 

  Also, we need to calculate the correlation coefficients between CO(GT) and other features to provide insight on the model to adopt. 

In [15]:
# show data distribution
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CO(GT),702.0,2.022578,1.102422,0.10000,1.20000,1.8000,2.750000,6.2500
PT08.S1(CO),764.0,1089.751963,178.174318,689.00000,966.00000,1058.2500,1199.500000,1784.0000
NMHC(GT),87.0,197.206897,149.160218,30.00000,78.25000,142.0000,269.500000,665.5000
C6H6(GT),764.0,9.331348,5.502068,0.60000,5.10000,8.1500,12.825000,29.6500
PT08.S2(NMHC),764.0,925.636780,208.328159,455.00000,769.37500,903.5000,1078.750000,1540.0000
NOx(GT),709.0,231.384344,177.874093,17.00000,104.00000,183.0000,297.000000,1116.5000
PT08.S3(NOx),764.0,823.006545,204.873809,429.00000,686.87500,806.2500,935.125000,1678.0000
NO2(GT),709.0,109.943583,40.936715,11.00000,82.00000,105.5000,134.000000,271.5000
PT08.S4(NO2),764.0,1434.821335,299.296616,647.00000,1240.25000,1477.5000,1636.125000,2292.5000
PT08.S5(O3),764.0,1014.630236,345.327384,305.00000,767.37500,954.5000,1220.125000,2322.5000


In [18]:
features = ['PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T(C)', 'RH', 'AH']

# calculate the correlation to the target: 'CO(GT)'
for f in features:
    related = df['CO(GT)'].corr(df[f])
    print("%s: %f" % (f, related)) 

PT08.S1(CO): 0.860035
NMHC(GT): 0.853855
C6H6(GT): 0.901409
PT08.S2(NMHC): 0.891297
NOx(GT): 0.777291
PT08.S3(NOx): -0.704531
NO2(GT): 0.672084
PT08.S4(NO2): 0.525154
PT08.S5(O3): 0.864918
T(C): 0.049157
RH: 0.047603
AH: 0.041407


The correlation coefficients shows that there's a strong relationship between CO(GT) and the other features except T(C), RH and AH. This means we can typically apply a linear regression model, such as support vector machines. Although, we can also apply multi-label classification models, such a decision trees and random forests.

We will proceed to;
1. drop these unrelated features
2. handle missing/duplicate/extreme values
3. setup a linear regression models for our tasks

In [19]:
# drop unrelated columns: 'T(C)', 'RH' and 'AH' (corr < 0.5)
df = df.drop(['T(C)', 'RH', 'AH'], axis=1)

# preview the data
df.head(10)

CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  \
Date       IsPeak                                                           
01/01/2005 0         2.15       1102.0       NaN      7.70          885.0   
           1         1.90       1054.0       NaN      5.60          791.0   
01/02/2005 0         1.45       1048.5       NaN      5.10          769.5   
           1         4.05       1379.5       NaN     17.10         1213.0   
01/03/2005 0         0.60        772.5       NaN      1.10          507.5   
           1         1.45        893.5       NaN      5.10          770.0   
01/04/2004 0         1.70       1125.0     127.0      6.30          825.0   
           1         3.50       1385.0     482.0     15.95         1171.0   
01/04/2005 0         1.00        892.0       NaN      2.65          627.5   
           1         1.20        933.5       NaN      3.85          703.5   

                   NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  
Date       IsPeak                                                             
01/01/2005 0         297.0         765.0    133.5        1051.0       1206.0  
           1         217.0         830.0    116.0         967.0       1131.0  
01/02/2005 0         241.0         800.0    146.0         957.0       1272.5  
           1         708.5         531.0    187.5        1299.0       1703.0  
01/03/2005 0         134.0        1344.0     84.0         647.0        317.5  
           1         297.0         917.5    132.5         805.5        629.5  
01/04/2004 0         102.0        1002.0     73.0        1425.0        944.0  
           1         201.5         750.0    121.5        1811.5       1340.5  
01/04/2005 0         147.5         974.0    100.0         926.5        461.5  
           1         190.0         852.0    125.0         969.0        574.5

In [20]:
# get row count
len(df)

782

In [21]:
# show count of missing entries
df.isna().sum(axis=0)

CO(GT)            80
PT08.S1(CO)       18
NMHC(GT)         695
C6H6(GT)          18
PT08.S2(NMHC)     18
NOx(GT)           73
PT08.S3(NOx)      18
NO2(GT)           73
PT08.S4(NO2)      18
PT08.S5(O3)       18
dtype: int64

In [22]:
# handle missing/invalid entries
# step 1 - drop 'NMHC(GT)' as most values are missing
df = df.drop('NMHC(GT)', axis=1) 

# step 2 - drop rows with missing entries
df.dropna(how='any', axis=0, inplace=True)

# preview the data
df.head(10)

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
Date       IsPeak                                                          
01/01/2005 0         2.15       1102.0      7.70          885.0    297.0   
           1         1.90       1054.0      5.60          791.0    217.0   
01/02/2005 0         1.45       1048.5      5.10          769.5    241.0   
           1         4.05       1379.5     17.10         1213.0    708.5   
01/03/2005 0         0.60        772.5      1.10          507.5    134.0   
           1         1.45        893.5      5.10          770.0    297.0   
01/04/2004 0         1.70       1125.0      6.30          825.0    102.0   
           1         3.50       1385.0     15.95         1171.0    201.5   
01/04/2005 0         1.00        892.0      2.65          627.5    147.5   
           1         1.20        933.5      3.85          703.5    190.0   

                   PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  
Date       IsPeak                                                    
01/01/2005 0              765.0    133.5        1051.0       1206.0  
           1              830.0    116.0         967.0       1131.0  
01/02/2005 0              800.0    146.0         957.0       1272.5  
           1              531.0    187.5        1299.0       1703.0  
01/03/2005 0             1344.0     84.0         647.0        317.5  
           1              917.5    132.5         805.5        629.5  
01/04/2004 0             1002.0     73.0        1425.0        944.0  
           1              750.0    121.5        1811.5       1340.5  
01/04/2005 0              974.0    100.0         926.5        461.5  
           1              852.0    125.0         969.0        574.5

In [24]:
# check to confirm the following;
# 1. count of missing values in each column is zero
# 2. 'NMHC(GT)' column has been dropped
df.isna().sum(axis=0)

CO(GT)           0
PT08.S1(CO)      0
C6H6(GT)         0
PT08.S2(NMHC)    0
NOx(GT)          0
PT08.S3(NOx)     0
NO2(GT)          0
PT08.S4(NO2)     0
PT08.S5(O3)      0
dtype: int64

In [25]:
# show the shape 
df.shape

(672, 9)

In [26]:
# handle duplicate values
# step 1 - drop any duplicate rows
df = df.drop_duplicates()

# step 2 - confirm the shape didn't changed
df.shape 

(672, 9)

# TASK 1: CO concentration prediction
Predict the CO concentration (in mg/m3) based on, at least, the PT08.S1(CO) raw sensor readings, day of the week and time. 

Maybe temperature and humidity can play a role as well? 

Use CO(GT) as the ground truth.

## 4. Further Data preparation (specific for this task)
In this section, we shall perform the following tasks;

1. Data cleaning - Data cleaning corrects errors and fills in missing data as a step to ensure data quality

   TO-DO:  additional data clean-up specific to this task;
   - make a copy of the dataset for further analysis

2. Data segregation - Data segregation partitions the dataset into Train/Test or Train/Test/Validation datasets

   TO-DO: partition the dataset into two(2) parts;
   - partition the data (train 80% and test 20%)

In [27]:
# make copy for further analysis
df1 = df.copy()

# preview te data
df1.head(10)

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
Date       IsPeak                                                          
01/01/2005 0         2.15       1102.0      7.70          885.0    297.0   
           1         1.90       1054.0      5.60          791.0    217.0   
01/02/2005 0         1.45       1048.5      5.10          769.5    241.0   
           1         4.05       1379.5     17.10         1213.0    708.5   
01/03/2005 0         0.60        772.5      1.10          507.5    134.0   
           1         1.45        893.5      5.10          770.0    297.0   
01/04/2004 0         1.70       1125.0      6.30          825.0    102.0   
           1         3.50       1385.0     15.95         1171.0    201.5   
01/04/2005 0         1.00        892.0      2.65          627.5    147.5   
           1         1.20        933.5      3.85          703.5    190.0   

                   PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  
Date       IsPeak                                                    
01/01/2005 0              765.0    133.5        1051.0       1206.0  
           1              830.0    116.0         967.0       1131.0  
01/02/2005 0              800.0    146.0         957.0       1272.5  
           1              531.0    187.5        1299.0       1703.0  
01/03/2005 0             1344.0     84.0         647.0        317.5  
           1              917.5    132.5         805.5        629.5  
01/04/2004 0             1002.0     73.0        1425.0        944.0  
           1              750.0    121.5        1811.5       1340.5  
01/04/2005 0              974.0    100.0         926.5        461.5  
           1              852.0    125.0         969.0        574.5

In [28]:
# shuffle indexes to redistributed sample
df1 = df1.reindex(np.random.permutation(df1.index))

# split the dataset into 80% train and 20% test datasets
train_df1 = df1[:int(0.8 * df1.shape[0])]
test_df1 = df1[int(0.8 * df1.shape[0]):]

In [29]:
# preview the training data
train_df1.head(10)

,,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3)
Date,IsPeak,,,,,,,,,
26/05/2004,1,2.00,1130.0,14.90,1146.0,159.0,1064.0,135.0,1480.0,1284.0
08/03/2005,0,1.50,989.5,5.35,782.0,258.0,808.0,157.0,900.0,956.5
07/06/2004,0,1.70,952.5,9.05,940.5,167.0,902.5,95.0,1686.5,939.0
28/04/2004,1,3.80,1477.5,20.20,1301.5,222.5,644.5,142.5,2007.5,1672.5
14/03/2004,0,2.50,1416.0,10.70,1002.0,160.0,933.0,119.0,1617.0,1262.0
30/04/2004,0,2.60,1303.5,12.40,1063.0,162.0,725.5,94.0,1847.5,1220.5
25/08/2004,0,1.30,986.5,6.30,825.5,71.0,800.0,71.0,1539.5,759.5
20/09/2004,1,3.70,1273.5,19.05,1270.0,448.0,598.0,168.0,1825.0,1533.0
23/03/2005,1,3.90,1486.5,20.90,1321.0,573.5,429.5,176.5,1681.0,1791.5


In [30]:
# preview the test data
test_df1.head(10)

,,CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3)
Date,IsPeak,,,,,,,,,
24/12/2004,0,4.25,1352.0,14.60,1137.0,610.0,567.0,174.0,1234.0,1967.0
05/01/2005,0,2.00,1242.0,8.50,921.0,267.0,626.0,126.0,1261.0,1405.0
17/12/2004,0,3.15,1061.0,5.70,800.0,516.0,819.0,150.0,1232.0,862.0
15/02/2005,1,2.05,1060.0,8.10,903.5,347.0,736.0,183.0,998.0,1099.0
27/04/2004,1,2.45,1166.0,13.55,1101.5,138.0,820.0,111.0,1651.0,1053.0
12/03/2004,0,1.90,1085.5,6.15,817.0,133.0,1250.5,110.0,1376.5,824.0
21/03/2005,0,1.40,1101.0,5.10,768.0,180.0,706.5,108.5,1219.0,919.5
11/06/2004,0,1.40,975.5,8.95,936.0,108.0,896.0,97.0,1641.0,903.0
29/03/2004,1,1.50,981.5,5.65,795.0,113.0,1108.0,98.0,1282.5,673.5


## 5. Model definition and training
In this section, we shall create a linear regression model and train it with the training dataset containing 80% of the overall data.  Afterwards, we will perform testing to evaluate the model with the remaining 20% of the data

In [31]:
# define the feature and target variables for task 1
t1_feature = ['PT08.S1(CO)']
t1_target = ['CO(GT)']

In [40]:
# define training input and output variables
df1_train_x = np.array(train_df1[t1_feature])
df1_train_y = np.array(train_df1[t1_target])

# show the shape of the input and taret features
print(df1_train_x.shape), print(df1_train_y.shape)

(537, 1)
(537, 1)


(None, None)

In [41]:
# create a regression model instance and fit it
t1_model = LinearRegression().fit(df1_train_x, df1_train_y)

In [42]:
# define training input and output variables
df1_test_x = np.array(test_df1[t1_feature])
df1_test_y = np.array(test_df1[t1_target])

# show the shape of the input and taret features
print(df1_test_x.shape), print(df1_test_y.shape)

(135, 1)
(135, 1)


(None, None)

In [43]:
# make model predictions on new or unseen data  
t1_prediction = t1_model.predict(df1_test_x)

# show  target prediction
t1_prediction

array([[3.31526037],
       [2.75775107],
       [1.84039485],
       [1.83532659],
       [2.37256282],
       [1.96456738],
       [2.04312551],
       [1.40705808],
       [1.43746768],
       [1.36144368],
       [2.03298898],
       [2.2483903 ],
       [2.1875711 ],
       [2.34215323],
       [0.74058105],
       [1.46027488],
       [1.75676846],
       [2.97568652],
       [1.33610235],
       [0.48716773],
       [2.08873991],
       [3.57374195],
       [2.90219666],
       [2.82363853],
       [4.18700219],
       [3.43436463],
       [2.1723663 ],
       [1.39438741],
       [2.31681189],
       [0.77099065],
       [0.92557277],
       [2.20024177],
       [3.03650572],
       [1.1257693 ],
       [4.02228353],
       [2.1799687 ],
       [1.0649501 ],
       [1.84292899],
       [1.09029143],
       [1.61992527],
       [2.14702497],
       [1.35890955],
       [3.67764142],
       [1.81758766],
       [1.34877302],
       [2.46379162],
       [1.67060793],
       [2.453

## 6. Model evaluation
In this section, we will evaluate the performance of the model we trained the last section. we will use the following model performance metrics; 
1. Coefficient of determination - This tells us how well the model fits a line to the data 
2. Root mean square error - This tells us to what degree the model reduced the residual errors in the data

In [44]:
# evaluate model score using model score (MS)
# NB: MS values tending to one give high predictions 
MS = t1_model.score(df1_test_x, df1_test_y)

# show model metrics
print(f"coefficient of determination: {MS}")
print(f"intercept: {t1_model.intercept_}")
print(f"coefficients: {t1_model.coef_}")

coefficient of determination: 0.759574271490659
intercept: [-3.53703577]
coefficients: [[0.00506827]]


In [45]:
# evaluate  model score using root mean square error (RMSE)
# NB: RMSE values tending to zero give high predictions  
RMSE = sqrt(mean_squared_error(y_true = df1_test_y, y_pred = t1_prediction))

# show model metric
print(f"root mean square error: {RMSE}")

root mean square error: 0.567430184509584


# TASK 2: Air Quality Index creation and prediction
Define an Air Quality Index (based on adequate literature) by combining the ground-truth readings of several gases.

Then, use ML to predict your Air Quality Index from several raw sensor readings and other columns of interest (obviously without using the ground truth column).

## 4. Further Data preparation (specific for this task)
In this section, we shall perform the following tasks;

1. Data cleaning - Data cleaning corrects errors and fills in missing data as a step to ensure data quality.

   TO-DO:  additional data clean-up steps specific to this task;

   - make a copy of the dataset for further analysis

*[ Source: w3schools - https://www.w3schools.com/python/pandas/pandas_cleaning.asp ]*

2. Feature engineering - Feature engineering refers to the manipulation (including the addition, deletion, combination, mutation) of the dataset to improve machine learning model training, leading to better performance and greater accuracy. Effective feature engineering is based on sound knowledge of the business problem and the available data sources. 

   TO-DO: additional feature engineering steps specific to this task;

   - Feature selection - select relevant features for further processing based on the ground-truth readings of several gases
   - Feature construction - create new [ AirQualityIndex ] feature by combining the ground-truth readings of several gases
   - Feature extraction - drop low-level features that are unsuitable for learning
   - Feature encoding - create symbolic binarized values to represent [ Low ] or [ High ] air quality index categories
   
*[ Source: Domino Data Lab - https://domino.ai/data-science-dictionary/feature-engineering ]*

3. Data segregation - Data segregation partitions the dataset into Train/Test or Train/Test/Validation datasets

   TO-DO: partition the dataset into two(2) parts;
   - partition the data (train 80% and test 20%)

In [46]:
# make copyfor further analysis
df2 = df.copy()

# preview the data
df2.head()

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
Date       IsPeak                                                          
01/01/2005 0         2.15       1102.0       7.7          885.0    297.0   
           1         1.90       1054.0       5.6          791.0    217.0   
01/02/2005 0         1.45       1048.5       5.1          769.5    241.0   
           1         4.05       1379.5      17.1         1213.0    708.5   
01/03/2005 0         0.60        772.5       1.1          507.5    134.0   

                   PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  
Date       IsPeak                                                    
01/01/2005 0              765.0    133.5        1051.0       1206.0  
           1              830.0    116.0         967.0       1131.0  
01/02/2005 0              800.0    146.0         957.0       1272.5  
           1              531.0    187.5        1299.0       1703.0  
01/03/2005 0             1344.0     84.0         647.0        317.5

we wiil define the Air Quality Index (AQI) based on the combination of other ground-truth readings 

we make this definition based the following justifications
1. all ground-truth gases were recorded in the same metric (μg/m3), hence, no need for further processing to normalize the data
2. by this definition, we make the following assumptions

    (a) air quality is based on the ground-truth of gases emitted from vehicles only (other source were not taken into account)
    
    (b) air quality is based on the average ground-truth of gases emitted from vehicles only (hence, we expect it will be LOW during off-peak hours, and HIGH during peak hours)

In [47]:
# define the Air Quality Index (AQI) based on the combination of other ground-truth readings 
df2['AQI'] = df2['CO(GT)'] + df2['C6H6(GT)'] + df2['NOx(GT)'] + df2['NO2(GT)']

# drop constituent columns
df2 = df2.drop(['CO(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)'], axis=1)

# preview the data
df2.head(10)

PT08.S1(CO)  PT08.S2(NMHC)  PT08.S3(NOx)  PT08.S4(NO2)  \
Date       IsPeak                                                           
01/01/2005 0            1102.0          885.0         765.0        1051.0   
           1            1054.0          791.0         830.0         967.0   
01/02/2005 0            1048.5          769.5         800.0         957.0   
           1            1379.5         1213.0         531.0        1299.0   
01/03/2005 0             772.5          507.5        1344.0         647.0   
           1             893.5          770.0         917.5         805.5   
01/04/2004 0            1125.0          825.0        1002.0        1425.0   
           1            1385.0         1171.0         750.0        1811.5   
01/04/2005 0             892.0          627.5         974.0         926.5   
           1             933.5          703.5         852.0         969.0   

                   PT08.S5(O3)     AQI  
Date       IsPeak                       
01/01/2005 0            1206.0  440.35  
           1            1131.0  340.50  
01/02/2005 0            1272.5  393.55  
           1            1703.0  917.15  
01/03/2005 0             317.5  219.70  
           1             629.5  436.05  
01/04/2004 0             944.0  183.00  
           1            1340.5  342.45  
01/04/2005 0             461.5  251.15  
           1             574.5  320.05

In [48]:
# show data distribution
df2['AQI'].describe()

count     672.000000
mean      353.047545
std       213.404672
min        41.300000
25%       201.212500
50%       305.450000
75%       439.562500
max      1336.200000
Name: AQI, dtype: float64

In [49]:
# define binarized categories for AQI using the rule below;
# set AQI to 0 when it's below the mean, otherwise, set it to 1
# NB: we used mean() to include its effect on outliers
df2.loc[df2['AQI'].lt(df2['AQI'].mean()), 'AQI'] = 0
df2.loc[df2['AQI'].ge(df2['AQI'].mean()), 'AQI'] = 1

# preview the data
df2['AQI'].head(10)

Date        IsPeak
01/01/2005  0         1.0
            1         0.0
01/02/2005  0         1.0
            1         1.0
01/03/2005  0         0.0
            1         1.0
01/04/2004  0         0.0
            1         0.0
01/04/2005  0         0.0
            1         0.0
Name: AQI, dtype: float64

In [50]:
# shuffle the rows to get a redistributed sample
df2 = df2.reindex(np.random.permutation(df2.index))

# split the dataset into 80% train and 20% test datasets
train_df2 = df2[:int(0.8 * df2.shape[0])]
test_df2 = df2[int(0.8 * df2.shape[0]):]

In [51]:
# preview the test dataset
train_df2.head(10)

,,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),AQI
Date,IsPeak,,,,,,
15/03/2004,0,1239.0,863.5,991.5,1575.0,1156.5,0.0
27/04/2004,0,1120.0,973.5,900.5,1576.0,877.5,0.0
22/07/2004,0,1131.5,1037.0,712.5,1640.5,1169.5,0.0
05/05/2004,1,1155.0,1084.0,736.0,1873.0,1083.0,0.0
05/03/2005,1,1000.0,712.0,819.0,989.0,1001.0,0.0
04/11/2004,0,999.0,900.5,738.5,1435.5,969.0,0.0
13/04/2004,1,1170.0,998.0,820.0,1613.5,1202.5,0.0
12/07/2004,0,937.5,868.0,878.0,1532.0,764.0,0.0
01/05/2004,0,1117.0,948.0,886.0,1656.0,901.0,0.0


In [52]:
# preview the test dataset
test_df2.head(10)

,,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),AQI
Date,IsPeak,,,,,,
05/05/2004,0,975.0,805.5,985.5,1599.5,692.5,0.0
14/05/2004,1,1141.5,1084.5,797.5,1670.5,1309.0,0.0
17/02/2005,0,1022.0,758.0,807.0,976.0,785.5,0.0
30/09/2004,1,1463.5,1338.5,527.0,1815.0,1666.5,1.0
23/03/2005,0,1148.5,820.0,651.0,1276.5,1104.5,0.0
29/03/2005,0,1054.5,763.5,715.0,1257.0,970.0,0.0
06/11/2004,0,1252.0,1134.0,586.0,1569.0,1210.0,1.0
22/03/2004,0,1083.0,767.5,1061.0,1541.0,789.0,0.0
31/07/2004,0,1076.0,995.5,732.5,1707.5,912.5,0.0


## 5. Model definition and training
In this section, we shall create a decision tree regression model and train it with the training dataset containing 80% of the overall data. Afterwards, we will perform testing to evaluate the model with the remaining 20% of the data

In [53]:
# define the features and target variables for task 2
t2_features = ['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)']
t2_target = ['AQI']

In [54]:
# define training input and output variables
df2_train_x = np.array(train_df2[t2_features])
df2_train_y = np.array(train_df2[t2_target])

# show the shape of the input and taret features
print(df2_train_x.shape), print(df2_train_y.shape)

(537, 5)
(537, 1)


(None, None)

In [55]:
# setup a decision tree regression model instance and fit it
t2_model = DecisionTreeRegressor().fit(df2_train_x, df2_train_y)

In [56]:
# define testing input and output variables
df2_test_x = np.array(test_df2[t2_features])
df2_test_y = np.array(test_df2[t2_target])

# show the shape of the input and taret features
print(df2_test_x.shape), print(df2_test_y.shape)

(135, 5)
(135, 1)


(None, None)

In [58]:
# make predictions on new or unseen data  
t2_prediction = t2_model.predict(df2_test_x)

# show target predictions
t2_prediction

array([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.])

## 6. Model evaluation
In this section, we will evaluate the performance of the model we trained the last section. we will use the following model performance metrics; 
1. Accuracy score - This tells us how well the model fits a decision tree to the data 
2. Classification report - This tells us to what degree the model performed using probabilistic metrics such as precision, recall, f1-score and support

In [59]:
# measure model accuracy
accuracy_score(y_true = df2_test_y, y_pred = t2_prediction)

0.9185185185185185

In [60]:
# show model quality report
report = classification_report(df2_test_y, t2_prediction)
print(report)

              precision    recall  f1-score   support

         0.0       0.94      0.93      0.94        90
         1.0       0.87      0.89      0.88        45

    accuracy                           0.92       135
   macro avg       0.91      0.91      0.91       135
weighted avg       0.92      0.92      0.92       135



# 7. Conclusions
In conclusion, we would say our model performed well with the data, scoring approxomately 75% and 91% on accuracy in task 1 and 2 respectively.

Although, it would been better to perform *multi-class classification* in task 2 instead of *binary classification*. However, our decision to go through with *binary classification* was because we didn't have any justifiable metrics within our AQI definition to categorize the data into appropriete bin ranges for easy intepretation, and also to reduce model bias. Hence, we made it 'Air Index Quality' simple with 'binary classification', 0 - LOW, 1 - HIGH

--- 

This cell goes to the very bottom of your submitted notebok.
You are requried to link the sources and web-links that you have used for various parts of this coursework. 

Sources:

- Data Ingestion - https://www.w3schools.com/python/pandas/pandas_csv.asp
- Data cleaning - https://www.w3schools.com/python/pandas/pandas_cleaning.asp
- Feature engineering - https://domino.ai/data-science-dictionary/feature-engineering